In [43]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Bài 1:

In [36]:
from pyspark.sql import SparkSession

# Khởi tạo Spark Session
spark = SparkSession.builder.appName("Lab3_Bai1").getOrCreate()

# Đường dẫn
path = "/content/drive/My Drive/BigData/BTTH3/"

# Đọc các file CSV
df_orders = spark.read.csv(path + "Orders.csv", header=True, inferSchema=True, sep=";")
df_customers = spark.read.csv(path + "Customer_List.csv", header=True, inferSchema=True, sep=";")
df_items = spark.read.csv(path + "Order_Items.csv", header=True, inferSchema=True, sep=";")
df_products = spark.read.csv(path + "Products.csv", header=True, inferSchema=True, sep=";")
df_reviews = spark.read.csv(path + "Order_Reviews.csv", header=True, inferSchema=True, sep=";")

# Hiển thị Schema để kiểm tra
print("Schema Orders:")
df_orders.printSchema()

print("Schema Customers:")
df_customers.printSchema()

print("Schema Order Items:")
df_items.printSchema()

print("Schema Products:")
df_products.printSchema()

print("Schema Reviews:")
df_reviews.printSchema()

spark.stop()

Schema Orders:
root
 |-- Order_ID: string (nullable = true)
 |-- Customer_Trx_ID: string (nullable = true)
 |-- Order_Status: string (nullable = true)
 |-- Order_Purchase_Timestamp: timestamp (nullable = true)
 |-- Order_Approved_At: timestamp (nullable = true)
 |-- Order_Delivered_Carrier_Date: timestamp (nullable = true)
 |-- Order_Delivered_Customer_Date: timestamp (nullable = true)
 |-- Order_Estimated_Delivery_Date: timestamp (nullable = true)

Schema Customers:
root
 |-- Customer_Trx_ID: string (nullable = true)
 |-- Subscriber_ID: string (nullable = true)
 |-- Subscribe_Date: date (nullable = true)
 |-- First_Order_Date: date (nullable = true)
 |-- Customer_Postal_Code: string (nullable = true)
 |-- Customer_City: string (nullable = true)
 |-- Customer_Country: string (nullable = true)
 |-- Customer_Country_Code: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)

Schema Order Items:
root
 |-- Order_ID: string (nullable = true)
 |-

Bài 2:

In [37]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import countDistinct

# Khởi tạo Spark Session
spark = SparkSession.builder.appName("Lab3_Bai2").getOrCreate()

# Đường dẫn
path = "/content/drive/My Drive/BigData/BTTH3/"

# Đọc dữ liệu
df_orders = spark.read.csv(path + "Orders.csv", header=True, inferSchema=True, sep=";")
df_customers = spark.read.csv(path + "Customer_List.csv", header=True, inferSchema=True, sep=";")
df_items = spark.read.csv(path + "Order_Items.csv", header=True, inferSchema=True, sep=";")

# Tổng số đơn hàng (đếm số dòng trong bảng Orders)
total_orders = df_orders.count()

# Số lượng khách hàng (đếm Subscriber_ID không trùng trong bảng Customer)
total_customers = df_customers.select(countDistinct("Subscriber_ID")).collect()[0][0]

# Số lượng người bán (đếm Seller_ID không trùng trong bảng Order_Items)
total_sellers = df_items.select(countDistinct("Seller_ID")).collect()[0][0]

print(f"Tổng số đơn hàng: {total_orders}")
print(f"Tổng số khách hàng (Không trùng): {total_customers}")
print(f"Tổng số người bán (Không trùng): {total_sellers}")

spark.stop()

Tổng số đơn hàng: 99441
Tổng số khách hàng (Không trùng): 99382
Tổng số người bán (Không trùng): 3095


Bài 3:

In [38]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc

spark = SparkSession.builder.appName("Lab3_Bai3").getOrCreate()
path = "/content/drive/My Drive/BigData/BTTH3/"

# Đọc dữ liệu
df_orders = spark.read.csv(path + "Orders.csv", header=True, inferSchema=True, sep=";")
df_customers = spark.read.csv(path + "Customer_List.csv", header=True, inferSchema=True, sep=";")

# Join csv Orders và Customers
df_joined = df_orders.join(df_customers, "Customer_Trx_ID")

# Group by country và đếm số đơn hàng
orders_by_country = df_joined.groupBy("Customer_Country") \
.agg(count("Order_ID").alias("Total_Orders")) \
.orderBy(col("Total_Orders").desc())

print("Số lượng đơn hàng theo quốc gia:")
# Chỉ show được 20 dòng (default)
orders_by_country.show(orders_by_country.count())

spark.stop()

Số lượng đơn hàng theo quốc gia:
+----------------+------------+
|Customer_Country|Total_Orders|
+----------------+------------+
|         Germany|       41754|
|          France|       12848|
|     Netherlands|       11629|
|         Belgium|        5464|
|         Austria|        5043|
|     Switzerland|        3640|
|  United Kingdom|        3382|
|          Poland|        2139|
|         Czechia|        2034|
|           Italy|        2025|
|           Spain|        1651|
|        Portugal|        1336|
|          Sweden|         975|
|         Denmark|         905|
|          Serbia|         746|
|          Norway|         716|
|        Slovakia|         534|
|        Slovenia|         495|
|          Turkey|         485|
|          Greece|         412|
|       Lithuania|         351|
|          Latvia|         280|
|         Croatia|         254|
|         Estonia|         148|
|         Finland|          81|
|      Luxembourg|          68|
|         Andorra|          46|
+------

Bài 4:

In [39]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, count, col, to_timestamp

spark = SparkSession.builder.appName("Lab3_Bai4").getOrCreate()
path = "/content/drive/My Drive/BigData/BTTH3/"

df_orders = spark.read.csv(path + "Orders.csv", header=True, inferSchema=True, sep=";")

# Định dạng Timestamp trong file là 'yyyy-MM-dd HH:mm'
# Ép lại cột Timestamp để chắc chắn đúng
df_orders = df_orders.withColumn("Order_Purchase_Timestamp", to_timestamp(col("Order_Purchase_Timestamp")))

# Trích xuất Năm và Tháng
df_orders = df_orders.withColumn("Year", year("Order_Purchase_Timestamp")) \
                     .withColumn("Month", month("Order_Purchase_Timestamp"))

# Group by Năm, Tháng và đếm đơn hàng
orders_by_time = df_orders.groupBy("Year", "Month") \
    .agg(count("Order_ID").alias("Total_Orders")) \
    .orderBy(col("Year").asc(), col("Month").desc())

print("Số lượng đơn hàng theo Năm/Tháng:")
orders_by_time.show(orders_by_time.count())

spark.stop()

Số lượng đơn hàng theo Năm/Tháng:
+----+-----+------------+
|Year|Month|Total_Orders|
+----+-----+------------+
|2022|   12|           1|
|2022|   10|         324|
|2022|    9|           4|
|2023|   12|        5673|
|2023|   11|        7544|
|2023|   10|        4631|
|2023|    9|        4285|
|2023|    8|        4331|
|2023|    7|        4026|
|2023|    6|        3245|
|2023|    5|        3700|
|2023|    4|        2404|
|2023|    3|        2682|
|2023|    2|        1780|
|2023|    1|         800|
|2024|   10|           4|
|2024|    9|          16|
|2024|    8|        6512|
|2024|    7|        6292|
|2024|    6|        6167|
|2024|    5|        6873|
|2024|    4|        6939|
|2024|    3|        7211|
|2024|    2|        6728|
|2024|    1|        7269|
+----+-----+------------+



Bài 5:

In [44]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count

spark = SparkSession.builder.appName("Lab3_Bai5").getOrCreate()
path = "/content/drive/My Drive/BigData/BTTH3/"

df_reviews = spark.read.csv(path + "Order_Reviews.csv", header=True, inferSchema=True, sep=";")

# Xử lý NULL và giá trị ngoại lệ
df_reviews = df_reviews.filter(col("Review_Score").isin([1, 2, 3, 4, 5]))

# Điểm đánh giá trung bình toàn bộ
avg_score = df_reviews.select(avg("Review_Score")).collect()[0][0]
print(f"Điểm đánh giá trung bình chung: {avg_score:.2f}\n")

# Số lượng đánh giá theo từng mức (1 -> 5)
score_counts = df_reviews.groupBy("Review_Score") \
    .agg(count("Review_ID").alias("Count")) \
    .orderBy("Review_Score")

print("Số lượng đánh giá theo từng mức điểm:")
score_counts.show()

spark.stop()

Điểm đánh giá trung bình chung: 4.09

Số lượng đánh giá theo từng mức điểm:
+------------+-----+
|Review_Score|Count|
+------------+-----+
|           1|11424|
|           2| 3151|
|           3| 8179|
|           4|19141|
|           5|57328|
+------------+-----+



Bài 6:

In [41]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, sum, to_timestamp

spark = SparkSession.builder.appName("Lab3_Bai6").getOrCreate()
path = "/content/drive/My Drive/BigData/BTTH3/"

df_orders = spark.read.csv(path + "Orders.csv", header=True, inferSchema=True, sep=";")
df_items = spark.read.csv(path + "Order_Items.csv", header=True, inferSchema=True, sep=";")
df_products = spark.read.csv(path + "Products.csv", header=True, inferSchema=True, sep=";")

# Chuyển đổi timestamp (nếu cần)
df_orders = df_orders.withColumn("Order_Purchase_Timestamp", to_timestamp(col("Order_Purchase_Timestamp")))

# Lọc đơn hàng trong năm 2024
df_orders = df_orders.filter(year("Order_Purchase_Timestamp") == 2024)

# Join Orders -> Order_Items -> Products
# Join Orders với Items để lấy giá tiền
df_joined = df_orders.join(df_items, "Order_ID")
# Join với Products để lấy tên danh mục
df_full = df_joined.join(df_products, "Product_ID")

# Tính doanh thu (Price + Freight_Value)
df_full = df_full.withColumn("Total_Value", col("Price") + col("Freight_Value"))

# Group theo danh mục và tính tổng
df_full = df_full.groupBy("Product_Category_Name") \
    .agg(sum("Total_Value").alias("Total_Value")) \
    .orderBy(col("Total_Value").desc())

print("Doanh thu năm 2024 theo danh mục sản phẩm:")
df_full.show(df_full.count(), truncate=False)

spark.stop()

Doanh thu năm 2024 theo danh mục sản phẩm:
+---------------------------------------+------------------+
|Product_Category_Name                  |Total_Value       |
+---------------------------------------+------------------+
|Health_Beauty                          |885191.119999997  |
|Watches_Gifts                          |771986.750000001  |
|Bed_Bath_Table                         |650794.700000002  |
|Sports_Leisure                         |621999.3399999994 |
|Computers_Accessories                  |594771.0400000002 |
|Housewares                             |491576.9600000012 |
|Furniture_Decor                        |476466.1300000007 |
|Auto                                   |404210.5700000002 |
|Baby                                   |299052.5599999998 |
|Cool_Stuff                             |273910.0500000001 |
|Garden_Tools                           |259068.31999999983|
|Telephony                              |217452.1299999995 |
|Perfumery                              |2